In [102]:
import numpy as np
# import math as m


In [112]:
def get_N(fi: float, a=6378137, e2=0.00669437999013):
    """fi w stopniach. Zwraca promień przekroju poprzecznego (pierwszego wertykału)"""
    fi = np.deg2rad(fi)
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    return N 

def get_M(fi: float, a=6378137, e2=0.00669437999013):
    """fi w stopniach. Zwraca promień przekroju podłużnego (południkowego)"""
    fi = np.deg2rad(fi)
    M = (a*(1-e2)) / ( (1 - e2 * np.sin(fi) ** 2) ** 1.5 )
    return M

def get_c(fi: float, az: float, a=6378137, e2=0.00669437999013):
    """fi, az w stopniach. Zwraca stałą Clairuta dla linii geodezyjnej"""
    fi = np.deg2rad(fi)
    az = np.deg2rad(az)
    N = get_N(fi, a, e2)
    c = N * np.cos(fi) * np.sin(az)
    return c

def get_n_and_ds(s):
    """s w metrach. Zwraca ilość podziałów ortodromy n i delta s pojedynczego odc. Krotka. Useful in Kivioj's algo"""
    # dobre ds to mniej wiecej 1km - 1.5km. Im mniejsze, tym lepsze (ale z umiarem - błędy numeryczne)
    n_float = s/1000
    n_int = np.ceil(n_float)
    ds = s/n_int
    return (n_int, ds)

def kivioj(fi: float, lam: float, az: float, s, a=6378137, e2=0.00669437999013):
    """fi, lam, az w stopniach. s w metrach. Zwraca wsp(fi, lam) punktu oddalonego o s,  biorąc azymut pocz az.
    wszystkie zwracane wartości w stopniach. Zwracany Azymut to przedłużenie linii geodezyjnej, a nie az odwrotny!"""
    org_fi = np.deg2rad(fi)
    org_lam = np.deg2rad(lam)
    org_az = np.deg2rad(az)
    fi = org_fi
    lam = org_lam
    az = org_az
    n, ds = get_n_and_ds(s)
    
    # KTÓRE AZ I FI SĄ ZAWSZE POCZĄTKOWE, A KTÓRE ZMIENNE W ITERACJACH ???????????
    for i in range(0, int(n)):  # n - 1
        d_fi = (ds*np.cos(az)) / get_M(np.rad2deg(fi))
        d_az = ds*np.sin(org_az)*np.tan(fi) / get_N(np.rad2deg(fi))
        
        sr_fi = fi + 0.5*d_fi
        sr_az = az + 0.5*d_az
        
        # TUTAJ ds to nadal całe ds, czy tylko połowa ??????????
        d_fi = (ds*np.cos(sr_az)) / get_M(np.rad2deg(sr_fi))
        d_az = ds*np.sin(sr_az)*np.tan(sr_fi) / get_N(np.rad2deg(sr_fi))
        d_lam = (ds*np.sin(sr_az)) / (get_N(np.rad2deg(sr_fi)) * np.cos(sr_fi))
        
        fi += d_fi
        lam += d_lam
        az += d_az
        
        # AZYMUT + - 180 stopni (np.pi)
        
    fi = np.rad2deg(fi)
    lam = np.rad2deg(lam)
    az = np.rad2deg(az)
    pkt_K = (fi, lam, az)
    return pkt_K 


In [323]:
#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
from math import atan
from math import atan2
from math import cos
from math import radians
from math import sin
from math import sqrt
from math import tan

#--- MAIN ---------------------------------------------------------------------+

def vincenty(coord1,coord2,maxIter=200,tol=10**-12):

    #--- CONSTANTS ------------------------------------+

    a=6378137.0                             # radius at equator in meters (WGS-84)
    f=1/298.257223563                       # flattening of the ellipsoid (WGS-84)
    b=(1-f)*a

    phi_1,L_1,=coord1                       # (lat=L_?,lon=phi_?)
    phi_2,L_2,=coord2                  

    u_1=atan((1-f)*tan(radians(phi_1)))
    u_2=atan((1-f)*tan(radians(phi_2)))

    L=radians(L_2-L_1)

    Lambda=L                                # set initial value of lambda to L

    sin_u1=sin(u_1)
    cos_u1=cos(u_1)
    sin_u2=sin(u_2)
    cos_u2=cos(u_2)

    #--- BEGIN ITERATIONS -----------------------------+
    for i in range(0,maxIter):

        cos_lambda=cos(Lambda)
        sin_lambda=sin(Lambda)
        sin_sigma=sqrt((cos_u2*sin(Lambda))**2+(cos_u1*sin_u2-sin_u1*cos_u2*cos_lambda)**2)
        cos_sigma=sin_u1*sin_u2+cos_u1*cos_u2*cos_lambda
        sigma=atan2(sin_sigma,cos_sigma)
        sin_alpha=(cos_u1*cos_u2*sin_lambda)/sin_sigma
        cos_sq_alpha=1-sin_alpha**2
        cos2_sigma_m=cos_sigma-((2*sin_u1*sin_u2)/cos_sq_alpha)
        C=(f/16)*cos_sq_alpha*(4+f*(4-3*cos_sq_alpha))
        Lambda_prev=Lambda
        Lambda=L+(1-C)*f*sin_alpha*(sigma+C*sin_sigma*(cos2_sigma_m+C*cos_sigma*(-1+2*cos2_sigma_m**2)))

        # successful convergence
        diff=abs(Lambda_prev-Lambda)
        if diff<=tol:
            break

    u_sq=cos_sq_alpha*((a**2-b**2)/b**2)
    A=1+(u_sq/16384)*(4096+u_sq*(-768+u_sq*(320-175*u_sq)))
    B=(u_sq/1024)*(256+u_sq*(-128+u_sq*(74-47*u_sq)))
    delta_sig=B*sin_sigma*(cos2_sigma_m+0.25*B*(cos_sigma*(-1+2*cos2_sigma_m**2)-(1/6)*B*cos2_sigma_m*(-3+4*sin_sigma**2)*(-3+4*cos2_sigma_m**2)))

    m=b*A*(sigma-delta_sig)                 # output distance in meters    
    
    # a1
    a1_num = cos_u2*sin(Lambda)
    a1_denom = (cos_u1*sin_u2) - (sin_u1*cos_u2*cos(Lambda))
    if (a1_num > 0 and a1_denom > 0):
        a1 = atan(a1_num/a1_denom)
    elif (a1_num > 0 and a1_denom<0):
        a1 = atan(a1_num/a1_denom) + 2*np.pi
    else:
        a1 = atan(a1_num/a1_denom) + np.pi
#     a1 = atan(a1_num/a1_denom)
    
    
    # a2
    a2_num = cos_u1*sin(Lambda)
    a2_denom = (sin_u2*cos_u1*cos(Lambda)) - (cos_u2*sin_u1)
    if (a2_num > 0 and a2_denom > 0):
        a2 = atan(a2_num/a2_denom)
    elif (a2_num > 0 and a2_denom<0):
        a2 = atan(a2_num/a2_denom) + 2*np.pi
    else:
        a2 = atan(a2_num/a2_denom) + np.pi
        
    if a2<0:
        a2 = 2*np.pi-a2
    if a1<0:
        a1 = 2*np.pi-a1
    
    a1 = np.rad2deg(a1)
    a2 = np.rad2deg(a2)
    return (m, a1, a2)

In [324]:
get_N(0)

6378137.0

In [325]:
get_M(0)

6335439.327292892

In [326]:
get_c(0, 90)

6378137.0

In [327]:
np.deg2rad(80)


1.3962634015954636

In [328]:
np.rad2deg(1.3962634015954636)

80.0

In [329]:
kivioj(0, 0, 90, 18918538.849887617)

(1.0476454796971385e-14, 169.94812602061478, 90.0)

In [330]:
kivioj(0, 0, 45, 10000000)

(45.09684653555572, 89.86840958013579, 90.05810325055786)

In [331]:
vincenty([0,0], [1, 170])

(18918538.849887617, 83.94007440533744, 275.97770841934096)

In [332]:
vincenty([0,0], [45.09684653555579, 89.86840958013579])

(10000000.007648008, 44.99933639672517, 270.0578604615721)

In [350]:
vincenty([0,0], [-1, 0])

(110574.38855796008, 180.0, 180.0)

In [342]:
vincenty([52,21], [55, 31])

(741928.3587486794, 59.32740467703917, 67.37596423902382)

In [351]:
kivioj(52,21,59.32740467703917,741928.3587486794)

(55.00002124349137, 30.99998465709942, 67.3759525715011)